In [26]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [27]:
player_ratings = pd.read_csv('../../FUT_player_data.csv')
player_ratings.head()

,id,futbin_id,name,height,weight,age,club,league,nation,rarity,...,phsyicality_jumping,physicality_stamina,physicality_strength,physicality_aggression,goalkeeper_diving,goalkeeper_handling,goalkeeper_kicking,goalkeeper_positioning,goalkeeper_reflexes,goalkeeper_speed
0,18949,54231,Kylian Mbappé,182,73,24,73,16,18,16,...,88.0,99.0,87.0,73.0,NaN,NaN,NaN,NaN,NaN,99
1,18981,54251,Karim Benzema,185,81,35,607,350,18,164,...,99.0,99.0,99.0,90.0,NaN,NaN,NaN,NaN,NaN,97
2,18982,54249,Zinedine Zidane,185,77,51,112658,2118,18,171,...,87.0,94.0,92.0,83.0,NaN,NaN,NaN,NaN,NaN,92
3,18730,54005,Pelé,173,70,82,112658,2118,54,153,...,90.0,91.0,78.0,61.0,NaN,NaN,NaN,NaN,NaN,96
4,19001,54277,Robert Lewandowski,185,81,35,241,53,37,164,...,99.0,97.0,99.0,99.0,NaN,NaN,NaN,NaN,NaN,97


In [30]:
player_valuations = pd.read_csv('../../data/preprocessed_valuations.csv')
player_valuations.head()

,Unnamed: 0,name,last_season,average_value
0,0,A.J. Soares,2016.0,2.187500e+05
1,1,AJ Leitch-Smith,2017.0,1.214286e+05
2,2,Aaron Appindangoyé,2023.0,8.333333e+05
3,3,Aaron Bastiaans,2020.0,1.656250e+05
4,4,Aaron Boupendza,2020.0,4.266667e+06


In [31]:
# Global historical inflation rate. Min year is 2012
inflation_rates = {
    2012: 0.0373,
    2013: 0.0262,
    2014: 0.0235,
    2015: 0.0143,
    2016: 0.0155,
    2017: 0.0219,
    2018: 0.0244,
    2019: 0.0221,
    2020: 0.0193,
    2021: 0.0348,
    2022: 0.0827,
}

def adjust_for_inflation(value, year):
    adjusted_value = value
    for y in range(year + 1, max(inflation_rates.keys()) + 1):
        adjusted_value *= (1 + inflation_rates[y])
    return adjusted_value



In [32]:
player_valuations['adjusted_value'] = player_valuations.apply(lambda row: adjust_for_inflation(row['average_value'], row['last_season']), axis=1)


TypeError: 'float' object cannot be interpreted as an integer

,Unnamed: 0,name,last_season,average_value
0,0,A.J. Soares,2016.0,2.187500e+05
1,1,AJ Leitch-Smith,2017.0,1.214286e+05
2,2,Aaron Appindangoyé,2023.0,8.333333e+05
3,3,Aaron Bastiaans,2020.0,1.656250e+05
4,4,Aaron Boupendza,2020.0,4.266667e+06
